In [1]:
# ==============================================================================
# SCRIPT FINAL DE TREINAMENTO DE IA - v1.0 (Análise Comportamental)
# ==============================================================================

# Passo 0: Importar todas as ferramentas necessárias
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
import numpy as np

# Ignora avisos futuros para uma saída mais limpa
warnings.filterwarnings('ignore')

# ------------------------------------------------------------------------------
# PASSO 1: Carregar os Dados REAIS e COMPLETOS do seu Banco de Dados na Nuvem
# ------------------------------------------------------------------------------
print("--- Passo 1: Carregando dados da Mainnet do seu servidor na nuvem ---")

# ATENÇÃO: Substitua pelo IP público do seu servidor
SERVER_IP = "34.79.99.180" 
db_connection_str = f'postgresql://admin:supersecret@{SERVER_IP}:5433/mempool_data'
df = pd.DataFrame()

try:
    db_engine = create_engine(db_connection_str)
    # Selecionamos todas as colunas que nosso coletor mais avançado salva
    df = pd.read_sql('SELECT hash, "inputData", to_address, from_address, nonce, gas_price, gas_limit, value, event_timestamp, base_fee_per_gas FROM transactions', con=db_engine)
    print(f"Sucesso! Carregamos {len(df)} transações REAIS e enriquecidas para análise.")
except Exception as e:
    print(f"--- ERRO ao carregar dados do banco: {e} ---")

# ------------------------------------------------------------------------------
# PASSO 2: Engenharia de Features Preditivas (O Coração da IA)
# ------------------------------------------------------------------------------
if not df.empty:
    print("\n--- Passo 2: Engenharia de Features focada em comportamento ---")
    
    # Limpeza e conversão de tipos de dados para garantir que tudo seja numérico
    for col in ['gas_price', 'base_fee_per_gas', 'value', 'gas_limit', 'nonce']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(inplace=True) # Remove linhas onde a conversão falhou
    
    # FEATURE #1 (A Pista de Urgência): A "Gorjeta" para o Validador
    df['priority_fee'] = df['gas_price'] - df['base_fee_per_gas']
    # Garante que não há valores negativos por flutuações mínimas
    df['priority_fee'] = df['priority_fee'].apply(lambda x: max(x, 0))

    # FEATURE #2 (A Pista de Complexidade): Tamanho do "Script" da Transação
    df['input_data_size'] = df['inputData'].apply(len)

    # ALVO (TARGET): Definindo o que queremos prever.
    # Uma transação é "altamente competitiva" (1) se estiver no top 15% das gorjetas.
    # Esta é a nossa "verdade" para o treinamento.
    limiar_interesse = df['priority_fee'].quantile(0.85)
    df['target'] = (df['priority_fee'] > limiar_interesse).astype(int)

    print(f"Features criadas com sucesso!")
    print(f"Definimos nosso alvo: transações com gorjeta > {limiar_interesse:.0f} wei são 'altamente competitivas' (target=1).")
    print("Distribuição do Alvo:", df['target'].value_counts(), sep='\n')

# ------------------------------------------------------------------------------
# PASSO 3: Treinamento e Avaliação do Cérebro Preditivo
# ------------------------------------------------------------------------------
# Só prosseguimos se tivermos criado a coluna 'target' e tivermos as duas classes (0 e 1)
if 'target' in df.columns and len(df['target'].unique()) > 1:
    print("\n--- Passo 3: Treinando o Modelo de IA ---")
    
    # Nossas "pistas" para a IA. Note que NÃO usamos a 'priority_fee' aqui para evitar "trapaça".
    features = ['value', 'gas_limit', 'input_data_size']
    X = df[features]
    y = df['target']
    
    # Dividindo os dados para treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Criando e treinando o modelo de Floresta Aleatória
    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    print("\nTreinando o cérebro preditivo...")
    model.fit(X_train, y_train)
    print("Treinamento concluído!")

    # Fazendo previsões nos dados de teste que o modelo nunca viu
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("\n--- AVALIAÇÃO DO MODELO PREDITIVO FINAL ---")
    print(f"Acurácia do Modelo: {accuracy * 100:.2f}%")
    print("\nRelatório de Classificação Detalhado:")
    print(classification_report(y_test, predictions, zero_division=0))

    # Mostrando a importância de cada feature para a decisão do modelo
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    print("\nImportância de cada 'Pista' para a decisão do modelo:")
    print(feature_importances)
else:
    print("\nNão foram encontrados dados suficientes para treinar o modelo. Deixe o coletor rodar por mais tempo.")

--- Passo 1: Carregando dados da Mainnet do seu servidor na nuvem ---
Sucesso! Carregamos 72 transações REAIS e enriquecidas para análise.

--- Passo 2: Engenharia de Features focada em comportamento ---
Features criadas com sucesso!
Definimos nosso alvo: transações com gorjeta > 2282194374 wei são 'altamente competitivas' (target=1).
Distribuição do Alvo:
target
0    40
1     7
Name: count, dtype: int64

--- Passo 3: Treinando o Modelo de IA ---

Treinando o cérebro preditivo...
Treinamento concluído!

--- AVALIAÇÃO DO MODELO PREDITIVO FINAL ---
Acurácia do Modelo: 86.67%

Relatório de Classificação Detalhado:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.50      0.50      0.50         2

    accuracy                           0.87        15
   macro avg       0.71      0.71      0.71        15
weighted avg       0.87      0.87      0.87        15


Importância de cada 'Pista' para a decisão do modelo: